<a href="https://colab.research.google.com/github/Lufec/MLP_Hand_Motion/blob/main/Hand_Motion_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/Postures.csv")

In [ ]:
df.head(5)

,Class,User,X0,Y0,Z0,X1,Y1,Z1,X2,Y2,Z2,X3,Y3,Z3,X4,Y4,Z4,X5,Y5,Z5,X6,Y6,Z6,X7,Y7,Z7,X8,Y8,Z8,X9,Y9,Z9,X10,Y10,Z10,X11,Y11,Z11
0,1,0,54.263880,71.466776,-64.807709,76.895635,42.462500,-72.780545,36.621229,81.680557,-52.919272,85.2322638852917,67.7492195028673,-73.684130041833,59.1885757027887,10.6789364098231,-71.2977813147725,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
1,1,0,56.527558,72.266609,-61.935252,39.135978,82.538530,-49.596509,79.223743,43.254091,-69.982489,87.4508729469625,68.4008083028339,-70.703990925959,61.5874515532753,11.7799190329758,-68.827417756239,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
2,1,0,55.849928,72.469064,-62.562788,37.988804,82.631347,-50.606259,78.451526,43.567403,-70.658489,86.8353875680762,68.9079249764243,-71.1383441365739,61.6864271910576,11.7934398850428,-68.88931646056,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
3,1,0,55.329647,71.707275,-63.688956,36.561863,81.868749,-52.752784,86.320630,68.214645,-72.228461,61.5961571288978,11.2506481750465,-68.9564252307431,77.3872254123912,42.7178334810919,-72.0151462991019,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
4,1,0,55.142401,71.435607,-64.177303,36.175818,81.556874,-53.475747,76.986143,42.426849,-72.574743,86.3687480605765,67.9012603746826,-72.4446499648169,61.2754021959714,10.8411094568665,-69.2799064015993,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?


In [ ]:
df = df.replace('?', 0.0)

In [ ]:
df.describe()

,Class,User,X0,Y0,Z0,X1,Y1,Z1,X2,Y2,Z2
count,78095.000000,78095.000000,78095.000000,78095.000000,78095.000000,78095.000000,78095.000000,78095.000000,78095.000000,78095.000000,78095.000000
mean,2.983776,7.959229,50.346308,85.813150,-29.985096,49.595844,86.193751,-29.509579,48.612744,83.772387,-30.560906
std,1.421152,4.697754,32.695886,40.203448,34.361971,32.477961,40.452297,34.764460,33.605155,41.022710,35.120384
min,1.000000,0.000000,-108.552738,-98.233756,-126.770872,-111.685241,-96.142589,-166.006838,-106.886524,-100.789312,-129.595296
25%,2.000000,5.000000,29.295141,63.497746,-56.356593,28.755679,64.156450,-57.360408,25.173405,58.053733,-58.654339
50%,3.000000,9.000000,54.620245,86.526334,-30.864248,54.215714,87.543493,-30.185331,53.814592,86.459935,-32.356535
75%,4.000000,12.000000,72.488833,113.108673,-1.419462,71.763080,116.228881,-0.368080,71.561988,106.661720,-0.946134
max,5.000000,14.000000,190.017835,169.175464,113.345119,188.691997,170.209350,104.697852,188.760168,168.186466,104.590879


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78095 entries, 0 to 78094
Data columns (total 38 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Class   78095 non-null  int64  
 1   User    78095 non-null  int64  
 2   X0      78095 non-null  float64
 3   Y0      78095 non-null  float64
 4   Z0      78095 non-null  float64
 5   X1      78095 non-null  float64
 6   Y1      78095 non-null  float64
 7   Z1      78095 non-null  float64
 8   X2      78095 non-null  float64
 9   Y2      78095 non-null  float64
 10  Z2      78095 non-null  float64
 11  X3      78095 non-null  object 
 12  Y3      78095 non-null  object 
 13  Z3      78095 non-null  object 
 14  X4      78095 non-null  object 
 15  Y4      78095 non-null  object 
 16  Z4      78095 non-null  object 
 17  X5      78095 non-null  object 
 18  Y5      78095 non-null  object 
 19  Z5      78095 non-null  object 
 20  X6      78095 non-null  object 
 21  Y6      78095 non-null  object 
 22

In [ ]:
df = df.astype('float')

In [ ]:
dfAux.User.value_counts()

10.0    8910
11.0    7934
13.0    6705
8.0     6130
14.0    5854
0.0     5853
5.0     4782
12.0    4500
6.0     4376
9.0     3920
1.0     2932
2.0     2594
7.0      370
4.0      212
Name: User, dtype: int64

In [ ]:
df1 = df.loc[dfAux['User'] == 0.0].sample(frac = 1)
df2 = df.loc[dfAux['User'] == 5.0].sample(frac = 1)
df3 = df.loc[dfAux['User'] == 12.0].sample(frac = 1)

In [ ]:
dfTrain = pd.concat([df1Train,df2Train,df3Train,df4Train,df5Train]).sample(frac = 1)
dfTeste = pd.concat([df1Test,df2Test,df3Test,df4Test,df5Test]).sample(frac = 1)

In [ ]:
x_train = dfTrain.drop(columns = 'Class')
y_train = dfTrain['Class']

Normalizacao Z-Score

In [ ]:
x_mean = x_train.mean()
x_std = x_train.std()
x_normZ = (x_train-x_mean)/x_std

Normalizacao MaxMin (pode ser removido)

In [ ]:
x_max = x_train.max()
x_min = x_train.min()
x_normMM = (x_train - x_min)/(x_max-x_min)

PCA caso queira testar (pode ser removido)

In [ ]:
corr = np.corrcoef(x_normZ,rowvar = False)

In [ ]:
eigvals,eigvect = np.linalg.eig(corr)

In [ ]:
eigvals/sum(eigvals)

array([0.23972351, 0.11123055, 0.09830255, 0.09022379, 0.09530997,
       0.09264783, 0.02123596, 0.01805514, 0.01589898, 0.01670868,
       0.04307571, 0.03803444, 0.04069202, 0.03987698, 0.03898388])

In [ ]:
#pc_feat = ['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10','PC11','PC12','PC13']
#x_pc = eigvect[:,[0,1,2,3,4,5,6,7,14,15,16,17,18,19,20]]

In [ ]:
#x_dec = np.dot(x_normZ,x_pc)

Conjunto de Teste

In [ ]:
x_test = dfTeste.drop(columns = 'Class')
y_test = dfTeste['Class']

Z-Score

In [ ]:
x_testZ = (x_test-x_mean)/x_std

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

Aplicações dos algoritmos foram testados com dados só normalizados  e dados normalizados + PCA. Só normalizados por Z-Score gerou melhor resultado

In [ ]:
svm = SVC(C= 9,
          kernel='rbf',            # kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’       
          gamma='auto' ,           # se gamma='scale'(padrão) é passado, então ele usa 1 / (n_features * X.var ()) como valor de gama,
                                   # se 'auto', usa 1 / n_features.
          degree = 4,              # grau do polinomio para kernel poly
          coef0 = 2,               # usa em poly e sigmoid
          tol=0.01,                # Tolerância para critério de parada.
          shrinking = True,        # uso da heurística de encolhimento
          probability = False,      # usa 5-fold cross validation
          cache_size=2048,         # Especifique o tamanho do cache do kernel (em MB).
          verbose= False, 
          max_iter=-1,             # Limite rígido nas iterações no solver ou -1 para nenhum limite.
          )

svm.fit(x_normZ, y_train.values.ravel())

print("Treino\n")
predictions = svm.predict(x_normZ)
confMatSVM = confusion_matrix(y_train,predictions)
print(confMatSVM)
print(classification_report(y_train,predictions))

print("\n____________________________________________________\n")
print("\n Teste\n")
predictionsTestSvm = svm.predict(x_testZ)
confMatSVM = confusion_matrix(y_test,predictionsTestSvm)
print(confusion_matrix(y_test,predictionsTestSvm))
print(classification_report(y_test,predictionsTestSvm))


Treino

[[13535     0     0     0     0]
 [    0 12086     0     0     0]
 [    0     0 13610     0     0]
 [    0     0     0 11853     0]
 [    0     0     0     0 12528]]
              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00     13535
         2.0       1.00      1.00      1.00     12086
         3.0       1.00      1.00      1.00     13610
         4.0       1.00      1.00      1.00     11853
         5.0       1.00      1.00      1.00     12528

    accuracy                           1.00     63612
   macro avg       1.00      1.00      1.00     63612
weighted avg       1.00      1.00      1.00     63612


____________________________________________________


 Teste



NameError: ignored

MLP (usando SKLearn) não gerou resultados ideais.

In [ ]:
mlp1 = MLPClassifier(solver = 'adam',    #adam, sgd, lbfgs
                     activation ='relu',       #identity, logistic, tanh, relu
                     hidden_layer_sizes=(10,10,10), 
                     alpha = 0.01,
                     learning_rate = 'constant',    #constant, invscaling, adaptive . só para sgd
                     learning_rate_init = 0.005,    #so para sgd
                     power_t = 0.1,                 #so para sgd, usando invscaling
                     momentum = 0.9,                #so para sgd
                     nesterovs_momentum = True,      #so para sgd e se momentum > 0
                     beta_1 = 0.9,                  #so para adam
                     beta_2 = 0.99,                  #so para adam
                     epsilon = 1e-8,                #so para adam
                     tol = 1e-5,
                     max_iter=1000,
                     verbose = False, 
                     early_stopping= False,
                     validation_fraction= 0.2,      #so se early stopping = true
                     shuffle = True, 
                     n_iter_no_change = 20)

mlp1.fit(x_normZ,y_train.values.ravel())

print("Treino\n")
predictions = mlp1.predict(x_normZ)
confMat1MLP = confusion_matrix(y_train,predictions)
print(confMat1MLP)
print(classification_report(y_train,predictions))


print("\n__________________\n")
print("\n Teste\n")
predictionsTestMlp = mlp1.predict(x_testZ)
confMat2MLP = confusion_matrix(y_test,predictionsTestMlp)
print(confMat2MLP)
print(classification_report(y_test,predictionsTestMlp))

Treino

[[10976    53   179    33    39]
 [  114  7790   801  1885   690]
 [  210    69 10209   603   189]
 [  145   667  2192  7861   415]
 [   73   354   491   709  9653]]
              precision    recall  f1-score   support

         1.0       0.95      0.97      0.96     11280
         2.0       0.87      0.69      0.77     11280
         3.0       0.74      0.91      0.81     11280
         4.0       0.71      0.70      0.70     11280
         5.0       0.88      0.86      0.87     11280

    accuracy                           0.82     56400
   macro avg       0.83      0.82      0.82     56400
weighted avg       0.83      0.82      0.82     56400


__________________


 Teste

[[2728   15   61    8    8]
 [  18 1947  197  495  163]
 [  54   15 2553  146   52]
 [  47  170  559 1925  119]
 [  25   86  127  205 2377]]
              precision    recall  f1-score   support

         1.0       0.95      0.97      0.96      2820
         2.0       0.87      0.69      0.77      2820
   